In [37]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Convolution2D, MaxPooling2D, InputLayer, BatchNormalization
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [5]:
from random import seed
seed(35)

In [6]:
test_set = pd.read_csv('path_to_file', low_memory=False) #Пожалуйста, вставьте путь скачанного файла fashion-mnist_test
train_set = pd.read_csv('path_to_file', low_memory=False) #Пожалуйста, вставьте путь скачанного файла fashion-mnist_train

In [7]:
x_train = train_set.loc[:, train_set.columns != 'label'] 
x_test = test_set.loc[:, test_set.columns != 'label']
y_train = train_set['label']
y_test = test_set['label']
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
# масштабируем интенсивности пикселей в диапазон [0, 1]
x_train /= 255
x_test /= 255
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

In [8]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((60000, 784), (10000, 784), (60000, 10), (10000, 10))

In [9]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train,
                                            test_size=0.2, random_state=35)

In [16]:
x_test = x_test.values.reshape(10000, 28, 28, 1)
x_train = x_train.values.reshape(48000, 28, 28, 1)
x_val = x_val.values.reshape(12000, 28, 28, 1)

Сравним, как зависит качество нейронной сети от количества сверточных слоев.

In [25]:
#Сеть с двумя сверточными слоями

model = Sequential()
model.add(InputLayer(input_shape=(28,28, 1)))
model.add(Convolution2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [26]:
model.compile(
  loss='categorical_crossentropy',
  optimizer=tf.keras.optimizers.Adam(),
  metrics=['accuracy']
)

In [29]:
batch_size = 128
epochs = 20

In [30]:
trHistory = model.fit(
  x_train, y_train,
  batch_size=batch_size,
  epochs=epochs,
  verbose=1,
  validation_data=(x_val, y_val)
)

Epoch 1/20
375/375 [==============================] - 14s 39ms/step - loss: 0.1085 - accuracy: 0.9604 - val_loss: 0.2938 - val_accuracy: 0.9086
Epoch 2/20
375/375 [==============================] - 14s 38ms/step - loss: 0.1012 - accuracy: 0.9630 - val_loss: 0.2906 - val_accuracy: 0.9094
Epoch 3/20
375/375 [==============================] - 14s 39ms/step - loss: 0.0950 - accuracy: 0.9665 - val_loss: 0.3060 - val_accuracy: 0.9127
Epoch 4/20
375/375 [==============================] - 15s 39ms/step - loss: 0.0896 - accuracy: 0.9674 - val_loss: 0.3029 - val_accuracy: 0.9154
Epoch 5/20
375/375 [==============================] - 15s 39ms/step - loss: 0.0820 - accuracy: 0.9708 - val_loss: 0.3074 - val_accuracy: 0.9104
Epoch 6/20
375/375 [==============================] - 15s 39ms/step - loss: 0.0768 - accuracy: 0.9727 - val_loss: 0.3248 - val_accuracy: 0.9078
Epoch 7/20
375/375 [==============================] - 16s 43ms/step - loss: 0.0730 - accuracy: 0.9730 - val_loss: 0.3300 - val_accuracy:

In [31]:
two_layers_loss, two_layers_accuracy = model.evaluate(x_test, y_test, verbose=0)
print('Two layers CNC test accuracy:', two_layers_accuracy)

Two layers CNC test accuracy: 0.914900004863739


Сверточная нейронная сеть из двух скрытых слоев показывает показывает результаты, превосходящие логистическую регрессию и полносвязную нейпронную сеть, так как сверточные сети отличаются очень высокой способностью к распознаванию паттернов на изображениях, являясь самый популярный вариант для их обработки.

Добавим еще один слой.

In [33]:
#Сеть с тремя сверточными слоями 
model = Sequential()
model.add(InputLayer(input_shape=(28,28, 1)))
model.add(Convolution2D(16, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))


In [34]:
model.compile(
  loss='categorical_crossentropy',
  optimizer=tf.keras.optimizers.Adam(),
  metrics=['accuracy']
)

In [35]:
trHistory = model.fit(
  x_train, y_train,
  batch_size=batch_size,
  epochs=epochs,
  verbose=1,
  validation_data=(x_val, y_val)
)

Epoch 1/20
375/375 [==============================] - 8s 20ms/step - loss: 0.8255 - accuracy: 0.7002 - val_loss: 0.5935 - val_accuracy: 0.7846
Epoch 2/20
375/375 [==============================] - 8s 21ms/step - loss: 0.5465 - accuracy: 0.8026 - val_loss: 0.5205 - val_accuracy: 0.8088
Epoch 3/20
375/375 [==============================] - 8s 21ms/step - loss: 0.4835 - accuracy: 0.8261 - val_loss: 0.4501 - val_accuracy: 0.8317
Epoch 4/20
375/375 [==============================] - 8s 21ms/step - loss: 0.4442 - accuracy: 0.8396 - val_loss: 0.4280 - val_accuracy: 0.8422
Epoch 5/20
375/375 [==============================] - 8s 21ms/step - loss: 0.4162 - accuracy: 0.8497 - val_loss: 0.4050 - val_accuracy: 0.8514
Epoch 6/20
375/375 [==============================] - 8s 21ms/step - loss: 0.3916 - accuracy: 0.8583 - val_loss: 0.3934 - val_accuracy: 0.8572
Epoch 7/20
375/375 [==============================] - 8s 21ms/step - loss: 0.3733 - accuracy: 0.8644 - val_loss: 0.3628 - val_accuracy: 0.8660

In [ ]:
three_layer_loss, three_layer_accuracy = model.evaluate(x_test, y_test, verbose=0)
print('Three layer CNN Test accuracy :', three_laye_accuracy)

Качество на тестовой выборке значительно упало,так как модель переобучилась.

Добавим к сети из двух сверточных слоев BatchNormalization слои и посмотрим, улучшит ли это качество.


In [38]:
#Сеть с двуми серточными слоями и BatchNormalization слоями
#Сеть с двумя сверточными слоями

model = Sequential()
model.add(InputLayer(input_shape=(28,28, 1)))
model.add(Convolution2D(32, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))


In [39]:
model.compile(
  loss='categorical_crossentropy',
  optimizer=tf.keras.optimizers.Adam(),
  metrics=['accuracy']
)

In [40]:
batch_size = 128
epochs = 20

In [42]:
trHistory = model.fit(
  x_train, y_train,
  batch_size=batch_size,
  epochs=epochs,
  verbose=1,
  validation_data=(x_val, y_val)
)

Epoch 1/20
375/375 [==============================] - 21s 55ms/step - loss: 0.3665 - accuracy: 0.8675 - val_loss: 0.7108 - val_accuracy: 0.7548
Epoch 2/20
375/375 [==============================] - 22s 58ms/step - loss: 0.2792 - accuracy: 0.8977 - val_loss: 0.3438 - val_accuracy: 0.8717
Epoch 3/20
375/375 [==============================] - 22s 58ms/step - loss: 0.2306 - accuracy: 0.9143 - val_loss: 0.3595 - val_accuracy: 0.8703
Epoch 4/20
375/375 [==============================] - 22s 59ms/step - loss: 0.2016 - accuracy: 0.9256 - val_loss: 0.2952 - val_accuracy: 0.8993
Epoch 5/20
375/375 [==============================] - 23s 61ms/step - loss: 0.1728 - accuracy: 0.9361 - val_loss: 0.2626 - val_accuracy: 0.9057
Epoch 6/20
375/375 [==============================] - 24s 65ms/step - loss: 0.1533 - accuracy: 0.9440 - val_loss: 0.2954 - val_accuracy: 0.8973
Epoch 7/20
375/375 [==============================] - 24s 65ms/step - loss: 0.1313 - accuracy: 0.9517 - val_loss: 0.2837 - val_accuracy:

In [43]:
batch_layer_loss, batch_layer_accuracy = model.evaluate(x_test, y_test, verbose=0)
print('Batch layer CNC Test Accuracy:', batch_layer_accuracy)

Batch layer CNC: 0.8830000162124634


In [45]:
print('Two layers CNC:', two_layers_accuracy)
print('Batch layer CNC:', batch_layer_accuracy)


Two layers CNC: 0.914900004863739
Batch layer CNC: 0.8830000162124634


Добавление BatchNormalization слоев не помогло улучшить качество, значит, мы получаем данные примерно одного порядка и нормализовать их дополнительно не требуется.